# Comparação de Estratégias de Ensino — Teste Z para Duas Médias

**Cenário:**
Uma escola quer avaliar se há diferença no desempenho médio dos alunos entre duas estratégias de ensino de matemática.

**Dados:**
- Amostra **Estratégia A**: notas de 50 alunos.
- Amostra **Estratégia B**: notas de 50 alunos.

**Objetivo:**
Aplicar **teste Z para duas amostras independentes** e concluir se existe diferença significativa entre as médias (nível de significância \(\alpha = 0{,}05\)).

## Observações importantes
- O teste Z é apropriado quando o desvio-padrão populacional é conhecido **ou** quando as amostras são grandes. Aqui, usamos **n=50** em cada grupo e usamos o desvio-padrão amostral como aproximação.
- Pressupostos: amostras independentes; distribuição aproximadamente normal das médias (garantida pelo tamanho amostral).
- Interpretação: teste **bicaudal** (H0: μA = μB; H1: μA ≠ μB).

In [ ]:
# Imports básicos
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from math import erf, sqrt

# Funções auxiliares
def phi(z):
    """CDF da Normal Padrão usando erf."""
    return 0.5 * (1.0 + erf(z / sqrt(2.0)))

def p_value_two_sided(z):
    return 2 * (1 - phi(abs(z)))


## 1) Dados das amostras
As notas foram simuladas de forma reprodutível para representar dois grupos de 50 alunos cada. As notas estão no intervalo de 0 a 100.

In [ ]:
# Gerando dados reprodutíveis
rng = np.random.default_rng(42)

# Estratégia A: média ~ 68, desvio ~ 10
a = rng.normal(loc=68, scale=10, size=50)

# Estratégia B: média ~ 72, desvio ~ 10
b = rng.normal(loc=72, scale=10, size=50)

# Notas não podem ser negativas ou acima de 100: fazemos um "clip"
a = np.clip(a, 0, 100)
b = np.clip(b, 0, 100)

df = pd.DataFrame({
    'Estrategia': ['A']*len(a) + ['B']*len(b),
    'Nota': np.concatenate([a, b])
})
df.head()

## 2) Estatística descritiva

In [ ]:
resumo = df.groupby('Estrategia')['Nota'].agg(['count','mean','std','min','max']).round(2)
resumo

## 3) Visualização rápida das distribuições

In [ ]:
# Histograma Estratégia A
plt.figure()
plt.hist(a, bins=10)
plt.title('Histograma — Estratégia A')
plt.xlabel('Nota')
plt.ylabel('Frequência')
plt.show()

In [ ]:
# Histograma Estratégia B
plt.figure()
plt.hist(b, bins=10)
plt.title('Histograma — Estratégia B')
plt.xlabel('Nota')
plt.ylabel('Frequência')
plt.show()

## 4) Teste Z para diferença de médias (bicaudal)
**Hipóteses**  
\(H_0: \mu_A = \mu_B\)  
\(H_1: \mu_A \neq \mu_B\)  
\(\alpha = 0{,}05\)

In [ ]:
# Parâmetros
nA, nB = len(a), len(b)
meanA, meanB = float(np.mean(a)), float(np.mean(b))
stdA, stdB = float(np.std(a, ddof=1)), float(np.std(b, ddof=1))  # ddof=1 -> desvio amostral

# Erro-padrão da diferença (usando desvios amostrais como aproximação)
SE = np.sqrt((stdA**2)/nA + (stdB**2)/nB)

# Estatística Z
z = (meanA - meanB) / SE
p = p_value_two_sided(z)

# Intervalo de confiança 95% para (muA - muB)
zcrit = 1.96  # normal aproximada para 95%
diff = meanA - meanB
ci_low = diff - zcrit*SE
ci_high = diff + zcrit*SE

print('Resumo do Teste Z (duas amostras independentes)')
print('-'*46)
print(f'nA = {nA}, nB = {nB}')
print(f'média A = {meanA:.2f}, desvio A = {stdA:.2f}')
print(f'média B = {meanB:.2f}, desvio B = {stdB:.2f}')
print(f'Diferença (A - B) = {diff:.2f}')
print(f'SE = {SE:.4f}')
print(f'Z = {z:.3f}')
print(f'p-valor (bicaudal) = {p:.4f}')
print(f'IC 95% para (μA - μB): [{ci_low:.2f}, {ci_high:.2f}]')

## 5) Conclusão
Se **p \le 0,05**, rejeitamos \(H_0\) e concluímos que há diferença significativa entre as médias das estratégias. Se **p > 0,05**, **não** rejeitamos \(H_0\) e concluímos que **não** há evidência suficiente de diferença.

_Observação:_ com amostras de tamanho 50, o teste Z e o teste t tendem a produzir conclusões semelhantes. Usamos aqui o Z por orientação do enunciado.